|                       |Total number of cars |Cars per minute|
|-----------------------|---------------------|--------------|
| Traffic_Laramie_1.mp4 | 6 | 2.02|
| Traffic_Laramie_2.mp4 | 4 | 2.27|

# Brief description of the frame differencing and background subtraction techniques
Both the frame differencing and background subtraction techniques are used in video processing for the purpose of
movement detection. They compare pixels of 2 frames and calculate the difference between them. The regions with
significant differences (above some threshold) are considered regions with some motion.
While background subtraction uses some relatively persistent image as a background and each other frame of the
video as an image to compare with the background, the frame differencing compares each consecutive pair of
frames in the video.
Both techniques require preprocessing the frames to simplify the calculation of the difference between pixels and
focus more on the movement itself than the exact borders of the moving objects. Usually, that preprocessing includes
grayscaling an image and applying a Gaussian Blur filter. But it also may include other approaches, for example,
image dilation to minimize small inaccuracies.
For background subtraction, it is a key task to choose a proper background and change it accordingly depending on
the light conditions and other environment changes. The frame differencing does not require that, but it can be less
accurate in detecting slowly moving objects.
# Brief analysis of the application
OpenCV library is used as the main tool for the implementation of the required functionality. I define helper functions
to calculate and draw the Main Street border coordinates. Also, I extracted a function for preprocessing a frame.
The main logic starts with reading a video file, the initial (background) frame, and its preprocessing. Then, in the
loop, it reads each next frame until no frames are left. The assessment says that the application must be based on
the frame differencing and background subtraction techniques. It is unclear if it is allowed to use only one of them.
Hence, my application uses both those techniques. For each iteration, I calculate 2 deltas: the first is between the
current frame and the background, and the second is between the current and previous frames. Each of those deltas
contributes to the resulting delta with 0.5 of weight. After calculating a threshold on the delta frame, I use
findContours from the OpenCV library to detect object boundaries. Objects with a total area of less than 3000 are
ignored. The found objects are framed into green rectangles.
The car’s counter is implemented by counting cars that cross a manually selected barrier. For that purpose, I check if
the top left angle of any of the detected rectangles is located on the left from and, at the same time, close to the
barrier. To avoid duplicates, I count such events only if there was a rectangle on the right from the barrier before.
That is not the most beautiful solution, but it works pretty well.

Start with installing required dependencies.

In [1]:
!pip install numpy opencv-python

Import required dependencies.

In [2]:
import cv2
import numpy as np
import os

Define a method to draw a red rectangle in open cv frame as shown in the screenshot from assessment.

In [3]:
def draw_dashed_rectangle(frame, start_point, end_point, color = (0, 0, 255), thickness = 4, dash_length = 15):
    start_x, start_y = start_point
    end_x, end_y = end_point

    for x in range(start_x, end_x, dash_length * 2):
        start_point = (x, start_y)
        end_point = (min(x + dash_length, end_x), start_y)
        cv2.line(frame, start_point, end_point, color, thickness)

        start_point = (x, end_y)
        end_point = (min(x + dash_length, end_x), end_y)
        cv2.line(frame, start_point, end_point, color, thickness)

    for y in range(start_y, end_y, dash_length * 2):
        start_point = (start_x, y)
        end_point = (start_x, min(y + dash_length, end_y))
        cv2.line(frame, start_point, end_point, color, thickness)

        start_point = (end_x, y)
        end_point = (end_x, min(y + dash_length, end_y))
        cv2.line(frame, start_point, end_point, color, thickness)

Define a method returning coordinates of the Main Street borders.

In [4]:
def get_main_street_border_coordinates(frame):
    return (
        2,
        int(frame.shape[0] * 0.43),
        frame.shape[1] - 4,
        frame.shape[0] - 10
    )

Define a method for a frame preprocessing. We crop the main street, apply grayscale and Gaussian Blur effects. Such a preprocessing simplifies car detection algorithm and improves its performance allowing to avoid unnecessary image details.

In [5]:
def preprocess_frame(frame, main_street_start_x, main_street_start_y, main_street_end_x, main_street_end_y):
    detection_frame = frame[main_street_start_y:main_street_end_y, main_street_start_x:main_street_end_x]
    gray_frame = cv2.cvtColor(detection_frame, cv2.COLOR_BGR2GRAY)
    return cv2.GaussianBlur(gray_frame, (25,25), 1)

Define a method to count cars given a vide file path.

In [6]:
def count_cars(video_file_path):

    # read video file
    video=cv2.VideoCapture(video_file_path)
    
    # find the video duration in minutes
    duration = int(video.get(cv2.CAP_PROP_FRAME_COUNT)) / video.get(cv2.CAP_PROP_FPS) / 60

    # read initial frame without cars for using in background subtraction
    initial_frame = video.read()[1]

    # define main street borders
    main_street_start_x, main_street_start_y, main_street_end_x, main_street_end_y = get_main_street_border_coordinates(initial_frame)

    # preprocess initial frame
    initial_frame = preprocess_frame(initial_frame, main_street_start_x, main_street_start_y, main_street_end_x, main_street_end_y)

    # prepare previous frame for using in frame differencing 
    prev_frame = initial_frame

    # read the next frame
    check, frame = video.read()
    
    # define a barrier line coordinates
    # crossing this line is counted as moving from the city’s downtown to the city centre
    count_barrier_start_x = main_street_start_x + 600
    count_barrier_start_y = main_street_start_y + 100
    count_barrier_end_x = count_barrier_start_x
    count_barrier_end_y = main_street_start_y + 170

    # init a flag to not count already counted car
    prev_car_before_barrier = False

    # init the current count to 0
    count = 0

    while check:

        # preprocess the current frame
        blur_frame = preprocess_frame(frame, main_street_start_x, main_street_start_y, main_street_end_x, main_street_end_y)

        # calculate "delta" frame using frame differencing and background subtraction techniques at the same time
        delta_frame = (cv2.absdiff(initial_frame, blur_frame) * 0.5 + cv2.absdiff(prev_frame, blur_frame) * 0.5).astype(np.uint8)

        # apply threshold to the "delta" frame for ease of objects detection
        threshold_frame=cv2.threshold(delta_frame, 12, 255, cv2.THRESH_BINARY)[1]

        # find objects contours in the threshold frame
        contours, _ = cv2.findContours(threshold_frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

        # iterate over found objects
        for c in contours:

            # skip small objects (people, bicycles, etc.)
            if cv2.contourArea(c) < 3000:
                continue

            # find coordinates of a rectangle around the object
            (x, y, w, h) = cv2.boundingRect(c)

            # draw rectangle as shown in the screenshot from assessment
            cv2.rectangle(
                frame,
                (x + main_street_start_x, y + main_street_start_y),
                (x+ main_street_start_x + w, y+main_street_start_y+h),
                (0,255,0),
                1
            )
            
            # count cars
            # if the car just crossed the line and there was a detected car before the line (basically the same car)
            if count_barrier_end_x - 50 < main_street_start_x + x  < count_barrier_start_x \
                and y + main_street_start_y < count_barrier_end_y \
                and prev_car_before_barrier:
                    # increment counter
                    count += 1
                    # set flag for car detected before the line to False
                    prev_car_before_barrier = False
            # if there is a car right before the line
            if count_barrier_start_x < main_street_start_x + x < count_barrier_end_x + 50 and y + main_street_start_y < count_barrier_end_y:
                # set flag for car detected before the line to True
                prev_car_before_barrier = True

        # draw the barrier line for better visual understanding
        cv2.line(frame, (count_barrier_start_x, count_barrier_start_y), (count_barrier_end_x, count_barrier_end_y), (255, 0, 0), 4)

        # draw the street text label as shown in the screenshot from assessment
        cv2.putText(frame, 'Main street', (main_street_start_x, main_street_start_y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 3)

        # draw counter
        cv2.putText(frame, str(count), (main_street_end_x - 60, main_street_start_y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,255), 3)

        # draw Main Street borders as shown in the screenshot from assessment
        draw_dashed_rectangle(
            frame = frame,
            start_point = (main_street_start_x, main_street_start_y),
            end_point = (main_street_end_x, main_street_end_y)
        )

        # show detected cars and counter
        cv2.imshow('Traffic Camera', frame)

        # update the previous frame
        prev_frame = blur_frame

        # read the next frame
        check, frame = video.read()

        # handle quit
        if cv2.waitKey(1) == ord('q'):
            break
            
    # gracefully shutdown the demonstration
    video.release()
    cv2.destroyAllWindows()
    
    # return the count and the cars rate per minute
    return count, round(count / duration, 2)

Iterate over the video files, count cars and print the result.

In [7]:
for video_file in [file for file in os.listdir('Exercise1_Files') if file.endswith('.mp4')]:
    total_number_of_cars, cars_per_minute = count_cars(f'Exercise1_Files/{video_file}')
    print(f'File Name: {video_file}, Total number of cars: {total_number_of_cars}, Cars per minute: {cars_per_minute}')

File Name: Traffic_Laramie_2.mp4, Total number of cars: 4, Cars per minute: 2.27
File Name: Traffic_Laramie_1.mp4, Total number of cars: 6, Cars per minute: 2.02
